In [ ]:
!pip install omegaconf
!pip install efficientnet

In [ ]:
from pathlib import Path
import cv2
import dlib
import numpy as np
import argparse
from contextlib import contextmanager
from omegaconf import OmegaConf
from tensorflow.keras.utils import get_file
from tensorflow.keras.models import Model
from keras.layers import Dense, Activation, BatchNormalization, GlobalMaxPool2D

#from efficientnet.keras import EfficientNetB0, EfficientNetB3
from tensorflow.keras.applications import EfficientNetB0, EfficientNetB3
import IPython.display as display
from PIL import Image
from IPython.display import display as display_image


In [ ]:
## Function to draw bounding box around detected faces

def draw_label(image, point, label, font=cv2.FONT_HERSHEY_SIMPLEX,
               font_scale=0.8, thickness=1):
    size = cv2.getTextSize(label, font, font_scale, thickness)[0]
    x, y = point
    cv2.rectangle(image, (x, y - size[1]), (x + size[0], y), (255, 0, 0), cv2.FILLED)
    cv2.putText(image, label, point, font, font_scale, (255, 255, 255), thickness, lineType=cv2.LINE_AA)



In [ ]:
## Function to instantiate EfficientNet model

def get_model(IMG_SIZE):

    base_model = EfficientNetB0(include_top=False, input_shape=(IMG_SIZE, IMG_SIZE, 3), pooling="avg")
    features = base_model.output
    pred_gender = Dense(units=2, activation="sigmoid", name="pred_gender")(features)
    pred_age = Dense(units=8, activation="softmax", name="pred_age")(features)
    model = Model(inputs=base_model.input, outputs=[pred_gender, pred_age])
    return model

In [ ]:
# Path to pretrained model
weight_file = "EfficientNetB0_224_weights.04-0.96.hdf5"
margin = 0.4
image_dir = None

# dlib library for face detection
detector = dlib.get_frontal_face_detector()

# Instantiate model and load pretrained weights
model_name, img_size = Path(weight_file).stem.split("_")[:2]
img_size = int(img_size)
cfg = OmegaConf.from_dotlist([f"model.model_name={model_name}", f"model.img_size={img_size}"])
img_size = int(cfg.model.img_size)
model = get_model(img_size)
model.load_weights(weight_file, by_name=True)


In [ ]:
# Function to access webcam and capture video frames

def access_webcam():
    # Access the webcam
    cap = cv2.VideoCapture(0)

    if not cap.isOpened():
        raise RuntimeError("Failed to access the webcam")

    try:
        # Capture a frame from the webcam
        ret, frame = cap.read()
        if not ret:
            raise RuntimeError("Failed to capture image")

        # Preprocess the captured frame (optional)
        #frame = cv2.cvtColor(frame, cv2.COLOR_BGR2RGB)
        #frame = cv2.resize(frame, (224, 224))
        display_image(Image.fromarray(cv2.cvtColor(frame, cv2.COLOR_BGR2RGB)))
        return frame
    
    finally:
        cap.release()

In [ ]:
# Capture images and resize them for facial detection
img = access_webcam()
input_img = cv2.cvtColor(img, cv2.COLOR_BGR2RGB)
img_h, img_w, _ = np.shape(input_img)

# Detect faces using dlib library
detected = detector(input_img, 1)
faces = np.empty((len(detected), img_size, img_size, 3))

# Only if faces are detected
if len(detected) > 0:
    for i, d in enumerate(detected):
        x1, y1, x2, y2, w, h = d.left(), d.top(), d.right() + 1, d.bottom() + 1, d.width(), d.height()
        xw1 = max(int(x1 - margin * w), 0)
        yw1 = max(int(y1 - margin * h), 0)
        xw2 = min(int(x2 + margin * w), img_w - 1)
        yw2 = min(int(y2 + margin * h), img_h - 1)
        cv2.rectangle(img, (x1, y1), (x2, y2), (255, 0, 0), 2)
        # cv2.rectangle(img, (xw1, yw1), (xw2, yw2), (255, 0, 0), 2)
        faces[i] = cv2.resize(img[yw1:yw2 + 1, xw1:xw2 + 1], (img_size, img_size))

    # predict ages and genders of the detected faces
    results = model.predict(faces)
    predicted_genders = results[0]
    ages = np.arange(0, 8).reshape(8, 1)
    predicted_ages = results[1].dot(ages).flatten()
    print(predicted_genders, predicted_ages)

    # draw results
    for i, d in enumerate(detected):
        #label = "{}, {}".format(int(predicted_ages[i]),"M" if predicted_genders[i][0] < 0.5 else "F")
        #draw_label(img, (d.left(), d.top()), label)
        label = "{}, {}".format(int(predicted_ages[i]),
                                        "M" if predicted_genders[i][0] < 0.5 else "F")

cv2.imshow("result", img)
key = cv2.waitKey(-1) if image_dir else cv2.waitKey(30)


In [ ]:
display_image(Image.fromarray(cv2.cvtColor(img, cv2.COLOR_BGR2RGB)))